# Clone repository

In [ ]:
!git clone https://github.com/CjangCjengh/vits.git


In [3]:
%cd F:\shixi\CS\vits3\vits
%pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
%pip install -r requirements.txt
!sudo apt-get install espeak -y

f:\shixi\CS\vits3\vits
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://download.pytorch.org/whl/cu113
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314478 sha256=55df801529b113bc3ef6ea35fd66942f8bee55cafab60e86ce8c41abf08ea67a
  Stored in directory: c:\users\15800\appdata\local\pip\cache\wheels\95\1a\6d\75355e7a5c76ed48e2d6cde3b95c4828e83274b93f5392ac96
  Created wheel for zhconv: filename=zhconv-1.4.3-py2.py3-none-any.whl size=208853 sha256=0376e5a931a0e7bda46e7a40f64226a5aa62370cf658d051fb357d9cd676c589
  Stored in directory: c:\users\15800\appdata\local\pip\cache\wheels\4b\de\f2\a313254d1c1bb66249397841f299ba71a40bfb125f1fa83429
Successfully built jieba zhconv
  Attempting uninstall: pypinyin
    Found existing installation: pypinyin 0.47.1
    Uninstalling pypinyin-0.47.1:
      Successfully uninstalled pypin

'sudo' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Unpack dataset

In [ ]:
!sudo apt-get install p7zip-full
!7z x ../drive/MyDrive/dataset.zip

# Alignment

In [4]:
%cd monotonic_align
!python setup.py build_ext --inplace
%cd ..

f:\shixi\CS\vits3\vits\monotonic_align
running build_ext
copying build\lib.win-amd64-3.7\monotonic_align\core.pyd -> monotonic_align
f:\shixi\CS\vits3\vits


# Clean

In [5]:
!python preprocess.py --text_index 1 --text_cleaners japanese_cleaners2 --filelists filelists/shioriko_train_filelist.txt filelists/shioriko_val_filelist.txt filelists/shioriko_test_filelist.txt

START: filelists/shioriko_train_filelist.txt
START: filelists/shioriko_val_filelist.txt
START: filelists/shioriko_test_filelist.txt


# Train

In [13]:
!python train.py -c configs/japanese_base2.json -m japanese_base2_shioriko

^C


In [6]:
%cd F:\shixi\CS\vits3\vits

import logging

numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.WARNING)

import librosa

%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text.cleaners import japanese_cleaners2
from text import cleaned_text_to_sequence
from pypinyin import lazy_pinyin, Style

from scipy.io.wavfile import write

def get_text(text, hps):
    text_norm = cleaned_text_to_sequence(text)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm
# hps_ms = utils.get_hparams_from_file("./configs/vctk_base.json")
hps = utils.get_hparams_from_file("./configs/japanese_base2.json")
# net_g_ms = SynthesizerTrn(
#     len(symbols),
#     hps_ms.data.filter_length // 2 + 1,
#     hps_ms.train.segment_size // hps.data.hop_length,
#     n_speakers=hps_ms.data.n_speakers,
#     **hps_ms.model)

net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model)
_ = net_g.eval()

def tts(text):
  stn_tst = get_text(text, hps)
  with torch.no_grad():
    x_tst = stn_tst.unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.float().numpy()
  ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate))
  write("example.wav", 22050, audio)
def jtts(text):
  stn_tst = get_text(japanese_cleaners2(text), hps)
  with torch.no_grad():
    x_tst = stn_tst.unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.float().numpy()
  ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate))
  write("example.wav", 22050, audio)

f:\shixi\CS\vits3\vits


In [14]:
_ = utils.load_checkpoint("F:\\shixi\\CS\\vits3\\drive\\MyDrive\\japanese_base2_shioriko\\G_90000.pth", net_g, None)
#_ = utils.load_checkpoint("F:\\shixi\\CS\\vits3\\G_45000.pth", net_g, None)

INFO:root:Loaded checkpoint 'F:\shixi\CS\vits3\drive\MyDrive\japanese_base2_shioriko\G_90000.pth' (iteration 3601)


In [13]:
tts("toshoshItsuwa...... moo daremo inai NdesUka?")

In [12]:
jtts("ハァ……ハァ……ハァ……。")

In [15]:
jtts("列車は必ず次の駅へ。なら舞台は？私たちは？")

In [16]:
jtts("本日より正式な部員として、スクールアイドル同好会にお世話になります。改めてよろしくお願いします。")

In [20]:
jtts("をざお、すうおずあんふあれ")

In [19]:
jtts("いい！うう！げめんみんじゃおぢんじえん！")

In [17]:
jtts("お誕生日おめでとうございます、ランジュ。私たちは幼馴染ですが、わかりあっていないところがありました……これからはもっとたくさん話をして、わかりあいましょう、あなたの困ったところも含めて、大好きです。")

In [18]:
jtts("吾輩は猫である。名前はまだない")